Extract monthly climate variables (ERA5Land) for the rgi subregions

In [1]:
import ee
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np

In [2]:
#%%
# # # Trigger the authentication flow.
# ee.Authenticate()

# # # Initialize the library.
ee.Initialize()

In [3]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_climate = os.path.join(folder_AGVA, "Climate")

In [7]:
### get convex hull of the glacier in each O3Region

# load rgi outlines
asset_rgi = ee.FeatureCollection('projects/lzeller/assets/rgi_2km_o3regions')

# aggregate list of unique subregions
subregions = asset_rgi.aggregate_array("O3Region").distinct().sort().remove(0)

# define function that gets convex hull of glaciers
def get_chull(region):
  
    # get geometry of the glaciers
    rgio3 = asset_rgi.filter(ee.Filter.eq('O3Region', region)).geometry()

    # get convex hull of the region
    rgio3_chull = rgio3.convexHull();

    # create feature to return
    feature_return = ee.Feature(rgio3_chull, {"O3Region":region})
    
    return feature_return

# get the bounding geometries for each
o3_chulls = ee.FeatureCollection( subregions.map(lambda x : get_chull(x) ))

In [8]:
### load in ERA5Land monthly data
era5land_monthly = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')

# define a function that will grab the era5land data for a specific year/month
def get_year_month_era5(y1,y2,m1,m2):
    subset_era5 = era5land_monthly.filter(ee.Filter.calendarRange(y1, y2, 'year'))
    subset_era5 = subset_era5.filter(ee.Filter.calendarRange(m1, m2, 'month'))
    return subset_era5

# Create a function to calculate the mean value of a raster within a feature geometry
def calculate_mean(raster, feature):
    
    # Calculate the mean values for the current month
    mean = raster.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=100,
        maxPixels=1e13
    )
    
    # return the mean
    return mean

# Create a function to calculate the standard deviation of a raster within a feature geometry
def calculate_std(raster, feature):
    
    # Calculate the mean values for the current month
    std = raster.reduceRegion(
        reducer=ee.Reducer.stdDev(),
        geometry=feature.geometry(),
        scale=100,
        maxPixels=1e13
    )
    
    # return the mean
    return std
        
# define function that will get mean ERA5 value of given variables, given a year, month, geometry, variables
# def get_ERA5_data(variables, year, month, feature):
    
#     # subset to just this year/month
#     subset_data = ee.Image(get_year_month_era5(year, year, month, month).first())
    
#     # select just the variable in question
#     subset_data = subset_data.select(variables)
    
#     # get the average of this data within the feature geometry
#     subset_average = calculate_mean(ee.Image(subset_data), feature)
    
#     return subset_average

def get_ERA5_daterange(variable, year_start, year_end, month_start, month_end, feature):
    
    # subset to just these years/months
    subset_data = get_year_month_era5(year_start, year_end, month_start, month_end)
    
    # select just the variable in question
    subset_data = subset_data.select(variable).toBands()
    
    # get the average of this data within the feature geometry
    subset_average = calculate_mean(ee.Image(subset_data), feature)
    
    # get the standard deviation of this data within the feature geometry
    subset_std = calculate_std(ee.Image(subset_data), feature)
    
    return subset_average


def get_ERA5_sum(variable, year_start, year_end, month_start, month_end, feature):
    
    # subset to just these years/months
    subset_data = get_year_month_era5(year_start, year_end, month_start, month_end)
    
    # select just the variable in question
    subset_data = subset_data.select(variable)
    
    # calculate sum across pixels
    subset_data = subset_data.sum()

In [26]:
# for each subregion, send it off to get the data
all_df = []
for region_n in [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]:
    print(f"Starting Region {region_n}")

    # get just this region geometry
    subset_chull = o3_chulls.filter(ee.Filter.eq('O3Region', region_n))

    all_p_avg = []
    all_t_avg = []
    all_p_std = []
    all_t_std = []
    
    # get winter precip in each year
    for y in [2018,2019,2020,2021,2022]:
        print("W",y)
        subset_era5 = era5land_monthly.filter(ee.Filter.calendarRange(y, y, 'year'))
        subset_era5 = subset_era5.filter(ee.Filter.calendarRange(1, 3, 'month')).select('total_precipitation_sum').sum()

        # get the mean, standard deviation of this data within the feature geometry
        std = calculate_std(ee.Image(subset_era5), subset_chull.first())
        avg = calculate_mean(ee.Image(subset_era5), subset_chull.first())
        all_p_std.append(std.getInfo()['total_precipitation_sum'])
        all_p_avg.append(avg.getInfo()['total_precipitation_sum'])
    
    # get summer temp in each year
    for y in [2018,2019,2020,2021,2022]:
        print("S",y)
        subset_era5 = era5land_monthly.filter(ee.Filter.calendarRange(y, y, 'year'))
        subset_era5 = subset_era5.filter(ee.Filter.calendarRange(6, 8, 'month')).select('temperature_2m').mean()

        # get the mean, standard deviation of this data within the feature geometry
        std = calculate_std(ee.Image(subset_era5), subset_chull.first())
        avg = calculate_mean(ee.Image(subset_era5), subset_chull.first())
        all_t_std.append(std.getInfo()['temperature_2m'])
        all_t_avg.append(avg.getInfo()['temperature_2m'])
        
    df_out = pd.DataFrame({'region':[region_n,region_n,region_n,region_n,region_n],
                           'year':[2018,2019,2020,2021,2022],
                           'p_avg':all_p_avg, 'p_std':all_p_std,
                           't_avg':all_t_avg, 't_std':all_t_std})
    all_df.append(df_out)

Starting Region 2
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 3
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 4
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 5
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 6
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 7
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 8
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 9
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 10
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 11
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 12
W 2018
W 2019
W 2020
W 2021
W 2022
S 2018
S 2019
S 2020
S 2021
S 2022
Starting Region 13
W 2018
W 2

In [30]:
test = pd.concat(all_df)
out_path = os.path.join(folder_climate, "O3Regions", f'spatialvariability.csv')
# test.to_csv(out_path, index=False)

In [43]:
df_o3 = test.groupby('region').mean()
df_o3 = test.groupby('region').min()
df_o3['p_std_percent']=df_o3['p_std']/df_o3['p_avg'] *100
df_o3['t_avg']=df_o3['t_avg']/3 -273.15
df_o3

,year,p_avg,p_std,t_avg,t_std,p_std_percent
region,,,,,,
2,2018,0.211056,0.024836,7.171715,6.067552,11.767535
3,2018,0.170616,0.029885,6.093144,5.627511,17.516113
4,2018,0.158902,0.020835,6.042961,10.285023,13.112006
5,2018,0.086698,0.008340,5.431696,5.951509,9.619542
6,2018,0.141456,0.042017,2.649055,9.642518,29.703393
7,2018,0.129003,0.022155,5.024536,2.692091,17.173696
8,2018,0.341489,0.053159,7.357080,4.166331,15.566810
9,2018,0.502348,0.064227,7.950443,3.759569,12.785304
10,2018,0.435654,0.125456,6.551396,7.833279,28.797106


In [40]:
test

,region,year,p_avg,p_std,t_avg,t_std
0,2,2018,0.304832,0.038244,840.965145,6.091268
1,2,2019,0.521593,0.112151,851.806837,7.612198
2,2,2020,0.211056,0.024836,844.222950,6.868980
3,2,2021,0.346505,0.056662,841.774123,6.067552
4,2,2022,0.563677,0.127757,844.844978,6.760306
...,...,...,...,...,...,...
0,16,2018,0.421411,0.158653,852.325638,5.368075
1,16,2019,0.384806,0.133559,850.870523,5.307289
2,16,2020,0.695538,0.286188,844.170797,5.044516
3,16,2021,0.859633,0.327369,851.031722,5.183877
